In [48]:
import random
import string
import time
from scipy import sparse

In [7]:
list_of_strs = [[string.ascii_lowercase[random.randrange(0, 26)] for i in range(random.randrange(5, 20))] for j in range(1000)]

In [11]:
list_of_chars = set.union(*[set(i) for i in list_of_strs])

In [36]:
char_dict = {value:idx for idx, value in enumerate(sorted(list_of_chars))}

In [39]:
char_dict[' '] = max(char_dict.values())+1

In [40]:
max_length = max([len(i) for i in list_of_strs])

In [41]:
def onehot_string(x, max_length):
    
    if x!=x:
        return None
    
    #Pad to max length
    while len(x)<max_length:
        x+=' '
        
    sparse_matrix = sparse.hstack([sparse.csr_matrix([0 if char_dict[current_char]!=i else 1 for i in range(len(char_dict))]) for current_char in x])
    
    return sparse_matrix

In [49]:
start_time = time.time()
sparse_strs = sparse.vstack([onehot_string(str_, max_length) for str_ in list_of_strs])
end_time = time.time()